# Unpivot Agri Dataset

## Initial Data

In [180]:

import pandas as pd
import sqlite3

# Step 1: Read the data
data = pd.read_excel("./raw agriculture & economic/Datahub_Agri_Latest.xlsx", sheet_name="Database")

# Step 2: Check for a typo and fix the column name if necessary
if 'Latiude' in data.columns:
    data = data.rename(columns={'Latiude': 'Latitude'})  # Fix the typo for Latitude

# Step 3: Define the unit for each indicator
indicator_units = {
    'Area Planted': 'Ha',
    'Area Harvested': 'Ha',
    'Quantity Harvested': 'T',
    'Yield': 'T/Ha',
    'Smallholder Livestock': 'Number',
    'Livestock Farming Industry': 'Number',
    'Total Number of Livestock': 'Number',
    'Quantity': 'Kg',
    'Value': 'USD',
    'Share(%)': '%',
    'Growth(%)': '%',
    'Products': 'N/A',
    'Famers': 'Number',
    'Associates': 'Number',
    'SMEs': 'Number',
    'Company': 'Number'
}

# Step 4: Unpivot the data using pd.melt()
initial_agri_data = data.melt(
    id_vars=[
        'Series Code', 'Series Name', 'Sector', 'Sub-Sector (1)', 'Sub-Sector (2)', 
        'Province', 'Year', 'Source', 'Latitude', 'Longitude', 'Pro code', 
        'Markets'
    ], 
    value_vars=[
        'Area Planted', 'Area Harvested', 'Quantity Harvested', 'Yield', 
        'Smallholder Livestock', 'Livestock Farming Industry', 'Total Number of Livestock', 
        'Livestock Production', 'Famers', 'Associates', 'SMEs', 'Company', 
        'Quantity', 'Value', 'Share(%)', 'Growth(%)'
    ], 
    var_name='Indicator', 
    value_name='Indicator Value'
)

# Step 5: Add the 'Indicator Unit' column using the indicator_units dictionary
initial_agri_data['Indicator Unit'] = initial_agri_data['Indicator'].map(indicator_units)
initial_agri_data['Indicator Value'] = initial_agri_data['Indicator Value'].replace(['', ' ', 'N/A'], pd.NA)
initial_agri_data = initial_agri_data.dropna(subset=['Indicator Value'])
initial_agri_data['Year'] = initial_agri_data['Year'].astype(int)
initial_agri_data = initial_agri_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

initial_agri_data["Date"] = ""

initial_agri_data

C:\Users\Mr.Whatanak\AppData\Local\Temp\ipykernel_36816\1240104783.py:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  initial_agri_data = initial_agri_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Series Code,Series Name,Sector,Sub-Sector (1),Sub-Sector (2),Province,Year,Source,Latitude,Longitude,Pro code,Markets,Indicator,Indicator Value,Indicator Unit,Date
0,NaN,Rice Production,Agriculture,Production,Rice,Kampong Cham,2020,MAFF,11.594763,105.272988,3,NaN,Area Planted,135910.000,Ha,
1,NaN,Rice Production,Agriculture,Production,Rice,Kandal,2020,MAFF,11.292682,104.563756,8,NaN,Area Planted,89459.000,Ha,
2,NaN,Rice Production,Agriculture,Production,Rice,Phnom Penh,2020,MAFF,11.343424,104.545573,12,NaN,Area Planted,7136.000,Ha,
3,NaN,Rice Production,Agriculture,Production,Rice,Prey Veng,2020,MAFF,11.285718,105.192062,14,NaN,Area Planted,391439.000,Ha,
4,NaN,Rice Production,Agriculture,Production,Rice,Svay Rieng,2020,MAFF,11.045571,105.481724,20,NaN,Area Planted,190075.000,Ha,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28633,NaN,Chillies Actual Rents \n\n(KHR m/ha),Agriculture,Total Factor Productivity (TFP) of Agriculture,Chillies,NaN,2023,IFAD-CHILLIES-CPARD,NaN,NaN,NaN,NaN,Value,0.122,USD,
28634,NaN,Chillies Capital Depreciation \n\n(KHR m/ha),Agriculture,Total Factor Productivity (TFP) of Agriculture,Chillies,NaN,2023,IFAD-CHILLIES-CPARD,NaN,NaN,NaN,NaN,Value,1.548,USD,
28635,NaN,Chillies Seeds \n\n(KHR m/ha),Agriculture,Total Factor Productivity (TFP) of Agriculture,Chillies,NaN,2023,IFAD-CHILLIES-CPARD,NaN,NaN,NaN,NaN,Value,0.828,USD,
28636,NaN,Chillies Fertilizer \n\n(KHR m/ha),Agriculture,Total Factor Productivity (TFP) of Agriculture,Chillies,NaN,2023,IFAD-CHILLIES-CPARD,NaN,NaN,NaN,NaN,Value,2.710,USD,


## Paddy Rice Price

In [181]:
import pandas as pd
import sqlite3

# Step 1: Read the data
data = pd.read_excel("./raw agriculture & economic/Rice Price_v4a.xlsx", sheet_name="Whatanak Paddy Rice Price")

paddy_rice_price = data.melt(id_vars=["Variety"], var_name="Date", value_name="Indicator Value")

# Add Columns
paddy_rice_price["Series Code"] = "" 
paddy_rice_price["Series Name"] = "Rice Price" 
paddy_rice_price["Sector"] = "Agriculture" 
paddy_rice_price["Sub-Sector (1)"] = "Paddy Rice Price"
paddy_rice_price["Sub-Sector (2)"] = paddy_rice_price["Variety"].apply(
    lambda x: "Fragrant Rice" if x in ["Sen Kra Ob 01", "Pka Romdoul/Jasmine"] 
              else ("White Rice" if x in ["OM", "IR"] 
                    else "")
)
paddy_rice_price["Source"] = ""
paddy_rice_price["Latitude"] = ""
paddy_rice_price["Longtitude"] = ""
paddy_rice_price["Indicator"] = "Price"
paddy_rice_price["Indicator Unit"] = "Riels/Kg"

paddy_rice_price["Province"] = ""
paddy_rice_price["Markets"] = ""
paddy_rice_price["Pro code"] = ""


paddy_rice_price

,Variety,Date,Indicator Value,Series Code,Series Name,Sector,Sub-Sector (1),Sub-Sector (2),Source,Latitude,Longtitude,Indicator,Indicator Unit,Province,Markets,Pro code
0,Pka Romdoul/Jasmine,2023-01-02 00:00:00,NaN,,Rice Price,Agriculture,Paddy Rice Price,Fragrant Rice,,,,Price,Riels/Kg,,,
1,OM,2023-01-02 00:00:00,1000.000000,,Rice Price,Agriculture,Paddy Rice Price,White Rice,,,,Price,Riels/Kg,,,
2,IR,2023-01-02 00:00:00,NaN,,Rice Price,Agriculture,Paddy Rice Price,White Rice,,,,Price,Riels/Kg,,,
3,Sen Kra Ob 01,2023-01-02 00:00:00,1100.000000,,Rice Price,Agriculture,Paddy Rice Price,Fragrant Rice,,,,Price,Riels/Kg,,,
4,Pka Romdoul/Jasmine,2023-01-16 00:00:00,NaN,,Rice Price,Agriculture,Paddy Rice Price,Fragrant Rice,,,,Price,Riels/Kg,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,Sen Kra Ob 01,2025-01-22 00:00:00,NaN,,Rice Price,Agriculture,Paddy Rice Price,Fragrant Rice,,,,Price,Riels/Kg,,,
292,Pka Romdoul/Jasmine,2025-02-03 00:00:00,NaN,,Rice Price,Agriculture,Paddy Rice Price,Fragrant Rice,,,,Price,Riels/Kg,,,
293,OM,2025-02-03 00:00:00,796.666667,,Rice Price,Agriculture,Paddy Rice Price,White Rice,,,,Price,Riels/Kg,,,
294,IR,2025-02-03 00:00:00,730.000000,,Rice Price,Agriculture,Paddy Rice Price,White Rice,,,,Price,Riels/Kg,,,


## FOB Rice Price

In [182]:
import pandas as pd
import sqlite3

# Step 1: Read the data
data = pd.read_excel("./raw agriculture & economic/Rice Price_v4a.xlsx", sheet_name="Export-Rice-Price-at-Port-FOB", skiprows=3)

# Drop completely empty columns
data = data.dropna(axis=1, how='all')

fob_rice_price = data.melt(id_vars=["VARIETIES"], var_name="Date", value_name="Indicator Value")

fob_rice_price = fob_rice_price.rename(columns={"VARIETIES": "Variety"})

# Add Columns
fob_rice_price["Series Code"] = "" 
fob_rice_price["Series Name"] = "Rice Price"
fob_rice_price["Sector"] = "Agriculture" 
fob_rice_price["Sub-Sector (1)"] = "FOB Rice Price" 
fob_rice_price["Sub-Sector (2)"] = fob_rice_price["Variety"].apply(
    lambda x: "Fragrant Rice" if x in [
        "Fragrant Rice (SKO)",
        "Fragrant Rice (SRO)", "Organic Premium Jasmine Rice", 
        "Brown Rice (Jamine Rice)", "Jamine A1 Super", "Jamine A1 Extra"
    ] 
    else ("White Rice" if x in [
        "Malys Angkor (Pka Romdoul)", "White Rice (Soft Texture)",
        "White Rice (Hard Texture)", "Premium White Rice", "Parboiled Rice",
        "Organic White Rice", "Organic Parboiled Rice", "Brown Rice (White Rice)"
    ] 
    else "")
)
fob_rice_price["Source"] = ""
fob_rice_price["Latitude"] = ""
fob_rice_price["Longtitude"] = ""
fob_rice_price["Indicator"] = "Price"
fob_rice_price["Indicator Unit"] = "TON/$USD (STANDARD 5% BROKEN)"

fob_rice_price["Province"] = ""
fob_rice_price["Markets"] = ""
fob_rice_price["Pro code"] = ""

fob_rice_price

,Variety,Date,Indicator Value,Series Code,Series Name,Sector,Sub-Sector (1),Sub-Sector (2),Source,Latitude,Longtitude,Indicator,Indicator Unit,Province,Markets,Pro code
0,Malys Angkor (Pka Romdoul),2023-01-03 00:00:00,810.0,,Rice Price,Agriculture,FOB Rice Price,White Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
1,Fragrant Rice (SKO),2023-01-03 00:00:00,780.0,,Rice Price,Agriculture,FOB Rice Price,Fragrant Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
2,Fragrant Rice (SRO),2023-01-03 00:00:00,NaN,,Rice Price,Agriculture,FOB Rice Price,Fragrant Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
3,White Rice (Soft Texture),2023-01-03 00:00:00,NaN,,Rice Price,Agriculture,FOB Rice Price,White Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
4,White Rice (Hard Texture),2023-01-03 00:00:00,NaN,,Rice Price,Agriculture,FOB Rice Price,White Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,Organic Parboiled Rice,2025-01-22 00:00:00,1030.0,,Rice Price,Agriculture,FOB Rice Price,White Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
444,Brown Rice (Jamine Rice),2025-01-22 00:00:00,890.0,,Rice Price,Agriculture,FOB Rice Price,Fragrant Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
445,Brown Rice (White Rice),2025-01-22 00:00:00,580.0,,Rice Price,Agriculture,FOB Rice Price,White Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,
446,Jamine A1 Super,2025-01-22 00:00:00,NaN,,Rice Price,Agriculture,FOB Rice Price,Fragrant Rice,,,,Price,TON/$USD (STANDARD 5% BROKEN),,,


## European Union - Milled non parboiled - Indica - Long B

In [183]:
import pandas as pd
import sqlite3

# Step 1: Read the data
european_indica_long_b = pd.read_excel("./raw agriculture & economic/Rice Price_v4a.xlsx", sheet_name="European Union - Milled non par")

european_indica_long_b = european_indica_long_b.rename(columns={"Rice variety": "Variety"})
european_indica_long_b = european_indica_long_b.rename(columns={"Price (€/Tonne)": "Indicator Value"})
european_indica_long_b = european_indica_long_b.rename(columns={"Reference Period": "Date"})

european_indica_long_b['Indicator Value'] = pd.to_numeric(european_indica_long_b['Indicator Value'], errors='coerce')

# Add Columns
european_indica_long_b["Series Code"] = ""
european_indica_long_b["Series Name"] = "Rice Price"
european_indica_long_b["Sector"] = "Agriculture" 
european_indica_long_b["Sub-Sector (1)"] = "European Union Rice Price" 
european_indica_long_b["Sub-Sector (2)"] = "Fragrant Rice" 
european_indica_long_b["Source"] = ""
european_indica_long_b["Latitude"] = ""
european_indica_long_b["Longtitude"] = ""
european_indica_long_b["Indicator"] = "Price"
european_indica_long_b["Indicator Unit"] = "£/Tonne"

european_indica_long_b["Province"] = ""
european_indica_long_b["Markets"] = ""
european_indica_long_b["Pro code"] = ""

european_indica_long_b



,Date,Variety,Indicator Value,Series Code,Series Name,Sector,Sub-Sector (1),Sub-Sector (2),Source,Latitude,Longtitude,Indicator,Indicator Unit,Province,Markets,Pro code
0,2017-07-17,European Union - Milled non parboiled - Indica...,570,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
1,2017-08-14,European Union - Milled non parboiled - Indica...,570,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
2,2017-08-21,European Union - Milled non parboiled - Indica...,570,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
3,2017-08-28,European Union - Milled non parboiled - Indica...,570,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
4,2017-09-04,European Union - Milled non parboiled - Indica...,570,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,2024-09-30,European Union - Milled non parboiled - Indica...,1100,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
297,2024-11-25,European Union - Milled non parboiled - Indica...,1190,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
298,2024-12-02,European Union - Milled non parboiled - Indica...,1190,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,
299,2024-12-09,European Union - Milled non parboiled - Indica...,1190,,Rice Price,Agriculture,European Union Rice Price,Fragrant Rice,,,,Price,£/Tonne,,,


## European Union - Milled non parboiled - Indica - Average

In [184]:
import pandas as pd
import sqlite3

# Step 1: Read the data
european_indica_average = pd.read_excel("./raw agriculture & economic/Rice Price_v4a.xlsx", sheet_name="European Union - Milled non (2)")


european_indica_average = european_indica_average.rename(columns={"Rice variety": "Variety"})
european_indica_average = european_indica_average.rename(columns={"Price (€/Tonne)": "Indicator Value"})
european_indica_average = european_indica_average.rename(columns={"Reference Period": "Date"})

european_indica_average['Indicator Value'] = pd.to_numeric(european_indica_average['Indicator Value'], errors='coerce')

# Add Columns
european_indica_average["Series Name"] = "Rice Price" 
european_indica_average["Series Code"] = "" 
european_indica_average["Sector"] = "Agriculture" 
european_indica_average["Sub-Sector (1)"] = "European Union Rice Price" 
european_indica_average["Sub-Sector (2)"] = "Fragrant Rice" 
european_indica_average["Source"] = ""
european_indica_average["Latitude"] = ""
european_indica_average["Longtitude"] = ""
european_indica_average["Indicator"] = "Price"
european_indica_average["Indicator Unit"] = "£/Tonne"

european_indica_average["Province"] = ""
european_indica_average["Markets"] = ""
european_indica_average["Pro code"] = ""



european_indica_average['Indicator Value']


0       563.1185
1       549.8935
2       576.8514
3       568.6626
4       576.5300
         ...    
389    1136.0335
390    1304.1480
391    1185.8525
392    1200.0381
393    1309.3538
Name: Indicator Value, Length: 394, dtype: float64

## Merge Agriculture Data

In [185]:
import sqlite3

combined_agriculture_df = pd.concat([initial_agri_data, paddy_rice_price, fob_rice_price, european_indica_long_b, european_indica_average], axis=0)
combined_agriculture_df = combined_agriculture_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# combined_agriculture_df['Indicator Value'] = initial_agri_data['Indicator Value'].replace(['', ' ', 'N/A'], pd.NA)
# combined_agriculture_df['Indicator Value'] = pd.to_numeric(combined_agriculture_df['Indicator Value'], errors='coerce')

combined_agriculture_df["Year"] = combined_agriculture_df["Year"].apply(
    lambda x: str(int(x)) if pd.notna(x) else x
)


# Ensure the 'Date' column is in datetime format
combined_agriculture_df["Date"] = pd.to_datetime(combined_agriculture_df["Date"])
combined_agriculture_df["Date"] = combined_agriculture_df["Date"].dt.strftime('%Y-%m-%d')

# Update 'Year' column only for rows where 'Year' is missing
combined_agriculture_df["Year"] = combined_agriculture_df.apply(
    lambda row: pd.to_datetime(row["Date"]).year if pd.isna(row["Year"]) and pd.notna(row["Date"]) else row["Year"],
    axis=1
)

# Optionally, you can save the unpivoted data back to a new Excel file
combined_agriculture_df.to_excel("Unpivoted_Datahub_Agriculture.xlsx", index=False)

# Step 6: Connect to SQLite database
conn = sqlite3.connect("data.db")
table_name = "agriculture_data"
combined_agriculture_df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

combined_agriculture_df[combined_agriculture_df['Variety'] == "Malys Angkor (Pka Romdoul)"]

C:\Users\Mr.Whatanak\AppData\Local\Temp\ipykernel_36816\3473862145.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_agriculture_df = combined_agriculture_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Series Code,Series Name,Sector,Sub-Sector (1),Sub-Sector (2),Province,Year,Source,Latitude,Longitude,Pro code,Markets,Indicator,Indicator Value,Indicator Unit,Date,Variety,Longtitude
0,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,810.0,TON/$USD (STANDARD 5% BROKEN),2023-01-03,Malys Angkor (Pka Romdoul),
14,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,810.0,TON/$USD (STANDARD 5% BROKEN),2023-01-12,Malys Angkor (Pka Romdoul),
28,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,860.0,TON/$USD (STANDARD 5% BROKEN),2023-02-28,Malys Angkor (Pka Romdoul),
42,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,850.0,TON/$USD (STANDARD 5% BROKEN),2023-03-19,Malys Angkor (Pka Romdoul),
56,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,850.0,TON/$USD (STANDARD 5% BROKEN),2023-03-25,Malys Angkor (Pka Romdoul),
70,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,850.0,TON/$USD (STANDARD 5% BROKEN),2023-05-02,Malys Angkor (Pka Romdoul),
84,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,820.0,TON/$USD (STANDARD 5% BROKEN),2023-07-14,Malys Angkor (Pka Romdoul),
98,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,850.0,TON/$USD (STANDARD 5% BROKEN),2023-08-29,Malys Angkor (Pka Romdoul),
112,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,865.0,TON/$USD (STANDARD 5% BROKEN),2023-09-08,Malys Angkor (Pka Romdoul),
126,,Rice Price,Agriculture,FOB Rice Price,White Rice,,2023,,,NaN,,,Price,928.0,TON/$USD (STANDARD 5% BROKEN),2023-12-14,Malys Angkor (Pka Romdoul),


# Unpivot Economic Dataset

In [186]:

import pandas as pd

# Step 1: Read the data
data = pd.read_excel("./raw agriculture & economic/Datahub_Agri_Latest.xlsx", sheet_name="Sheet1")

# Step 2: Check for a typo and fix the column name if necessary
if 'Latiude' in data.columns:
    data = data.rename(columns={'Latiude': 'Latitude'})  # Fix the typo for Latitude

# Step 3: Define the unit for each indicator
indicator_units = {
    'Quantity': 'Kg',
    'Value': 'USD',
    'Share(%)': '%',
    'Growth(%)': '%'
}

# Step 4: Unpivot the data using pd.melt()
unpivoted_data = data.melt(
    id_vars=[
        'Series Code', 'Series Name', 'Sector', 'Sub-Sector (1)', 'Sub-Sector (2)', 
        'Province', 'Year', 'Source', 'Latitude', 'Longitude',
        'Products', 'Markets'
    ], 
    value_vars=[
        'Quantity', 'Value', 'Share(%)', 'Growth(%)'
    ], 
    var_name='Indicator', 
    value_name='Indicator Value'
)

# Step 5: Add the 'Indicator Unit' column using the indicator_units dictionary
unpivoted_data['Indicator Unit'] = unpivoted_data['Indicator'].map(indicator_units)
unpivoted_data['Indicator Value'] = unpivoted_data['Indicator Value'].replace(['', ' ', 'N/A'], pd.NA)
unpivoted_data = unpivoted_data.dropna(subset=['Indicator Value'])
unpivoted_data['Year'] = pd.to_numeric(unpivoted_data['Year'], errors='coerce')  # Convert to numeric, invalid values become NaN
unpivoted_data['Year'] = unpivoted_data['Year'].fillna(0).astype('int')
unpivoted_data['Year'] = unpivoted_data['Year'].apply(lambda x: int(x) if not pd.isna(x) else x)  # Convert non-NaN values to int
unpivoted_data['Year'] = unpivoted_data['Year'].astype('int')
unpivoted_data = unpivoted_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# Step 6: Strip leading and trailing spaces from text columns
text_columns = [
    'Series Name', 'Sector', 'Sub-Sector (1)',
    'Source', 'Products', 'Markets', 'Indicator', 'Indicator Unit'
]

for col in text_columns:
    unpivoted_data[col] = unpivoted_data[col].str.strip()
    
# Optionally, you can save the unpivoted data back to a new Excel file
unpivoted_data.to_excel("Unpivoted_Datahub_Economic.xlsx", index=False)

# Step 6: Connect to SQLite database
conn = sqlite3.connect("data.db")
table_name = "economic_data"
unpivoted_data.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()


C:\Users\Mr.Whatanak\AppData\Local\Temp\ipykernel_36816\2070881205.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  unpivoted_data = unpivoted_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# Unpivot Education Data

## Public education statistic by class

In [187]:
import pandas as pd

# Define file path
file_path = "./raw education/Public education statistic by class.xlsx"

# Load all sheet names
sheet_names = [
    sheet for sheet in pd.ExcelFile(file_path).sheet_names 
    if not any(substring.lower() in sheet.lower() for substring in ["Dropout_AllGrades_AllYears", "Primary_Lower&Upper Secondary"])
]

# Initialize an empty list to store DataFrames
df_list = []

# Loop through each sheet
for sheet in sheet_names:
    # Read data
    data = pd.read_excel(file_path, sheet_name=sheet, header=[1, 2])

    # Flatten MultiIndex columns
    data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in data.columns]

    # Identify the correct Province column name
    province_col = [col for col in data.columns if "Province" in col][0]  # Automatically find it

    # Add "Series Name" column with a fixed value
    data["Series Name"] = "Student Flow Rates By Class"

    # Melt the DataFrame
    unpivoted_data = data.melt(
        id_vars=[province_col, "Series Name"],  # Include "Series Name" in id_vars
        var_name='Grade_Indicator',
        value_name='Indicator Value'
    )

    # Split "Grade_Indicator" into "Grade" and "Indicator"
    unpivoted_data[['Grade', 'Indicator']] = unpivoted_data['Grade_Indicator'].str.rsplit('_', n=1, expand=True)

    # Rename province column back to "Province"
    unpivoted_data.rename(columns={province_col: "Province"}, inplace=True)

    # Drop the original "Grade_Indicator" column
    unpivoted_data.drop(columns=['Grade_Indicator'], inplace=True)

    # Add "Year" column from the sheet name
    unpivoted_data["Year"] = sheet

    # Append to list
    df_list.append(unpivoted_data)

# Combine all years into a single DataFrame
student_flow_rates = pd.concat(df_list, ignore_index=True)

# Add Columns
student_flow_rates["Series Code"] = "" 
student_flow_rates["Sector"] = "Education" 
student_flow_rates["Sub-Sector (1)"] = ""
student_flow_rates["Sub-Sector (2)"] = ""
student_flow_rates["Source"] = ""
student_flow_rates["Latitude"] = ""
student_flow_rates["Longtitude"] = ""
student_flow_rates["Indicator Unit"] = "%"

student_flow_rates
# # Filtering where Indicator is 'Promotion'
# promotion_df = student_flow_rates[student_flow_rates['Indicator'] == 'Promotion']
# promotion_df["Series Name"] = "Student Promotion Rates By Class"

# # Filtering where Indicator is 'Repitition'
# repitition_df = student_flow_rates[student_flow_rates['Indicator'] == 'Repitition']
# repitition_df["Series Name"] = "Student Repitition Rates By Class"
# repitition_df

# # Filtering where Indicator is 'Dropout'
# dropout_df = student_flow_rates[student_flow_rates['Indicator'] == 'Dropout']
# dropout_df["Series Name"] = "Student Dropout Rates By Class"
# dropout_df

,Province,Series Name,Indicator Value,Grade,Indicator,Year,Series Code,Sector,Sub-Sector (1),Sub-Sector (2),Source,Latitude,Longtitude,Indicator Unit
0,Banteay Meanchey,Student Flow Rates By Class,86.2,Grade 1,Promotion,2012-2013,,Education,,,,,,%
1,Battambang,Student Flow Rates By Class,80.9,Grade 1,Promotion,2012-2013,,Education,,,,,,%
2,Kampong Cham,Student Flow Rates By Class,87.9,Grade 1,Promotion,2012-2013,,Education,,,,,,%
3,Kampong Chhnang,Student Flow Rates By Class,82.4,Grade 1,Promotion,2012-2013,,Education,,,,,,%
4,Kampong Speu,Student Flow Rates By Class,85.0,Grade 1,Promotion,2012-2013,,Education,,,,,,%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10039,Takeo,Student Flow Rates By Class,27.0,Grade 12,Dropout,2022-2023,,Education,,,,,,%
10040,Tbaung Khmum,Student Flow Rates By Class,24.5,Grade 12,Dropout,2022-2023,,Education,,,,,,%
10041,Whole Kingdom,Student Flow Rates By Class,26.6,Grade 12,Dropout,2022-2023,,Education,,,,,,%
10042,- Urban Area,Student Flow Rates By Class,32.7,Grade 12,Dropout,2022-2023,,Education,,,,,,%


## Public education statistic by class range

### Flow Rate By Class Range

In [188]:
import pandas as pd

# Define file path
file_path = "./raw education/Public education statistic by class range.xlsx"

# Load all sheet names
sheet_names = [sheet for sheet in pd.ExcelFile(file_path).sheet_names if "dropout_trend" not in sheet.lower()]

# Initialize an empty list to store DataFrames
df_list = []

# Loop through each sheet
for sheet in sheet_names:
    # Read data
    data = pd.read_excel(file_path, sheet_name=sheet, header=[2, 3])
    data = data.iloc[:, :10]

    # Flatten MultiIndex columns
    data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in data.columns]
    

    # Identify the correct Province column name
    province_col = [col for col in data.columns if "Provinces" in col][0]  # Automatically find it

    # Add "Series Name" column with a fixed value
    data["Series Name"] = "Student Flow Rates By Class Range"

    # Melt the DataFrame
    unpivoted_data = data.melt(
        id_vars=[province_col, "Series Name"],  # Include "Series Name" in id_vars
        var_name='Grade_Indicator',
        value_name='Indicator Value'
    )

    # Split "Grade_Indicator" into "Grade" and "Indicator"
    unpivoted_data[['Grade', 'Indicator']] = unpivoted_data['Grade_Indicator'].str.rsplit('_', n=1, expand=True)

    # Rename province column back to "Province"
    unpivoted_data.rename(columns={province_col: "Provinces"}, inplace=True)

    # Drop the original "Grade_Indicator" column
    unpivoted_data.drop(columns=['Grade_Indicator'], inplace=True)

    # Add "Year" column from the sheet name
    unpivoted_data["Year"] = sheet

    # Append to list
    df_list.append(unpivoted_data)

# Combine all years into a single DataFrame
student_flow_rates_and_graduates = pd.concat(df_list, ignore_index=True)

# Add Columns
student_flow_rates_and_graduates["Series Code"] = "" 
student_flow_rates_and_graduates["Sector"] = "Education" 
student_flow_rates_and_graduates["Sub-Sector (1)"] = "" 
student_flow_rates_and_graduates["Sub-Sector (2)"] = "" 
student_flow_rates_and_graduates["Source"] = ""
student_flow_rates_and_graduates["Latitude"] = ""
student_flow_rates_and_graduates["Longtitude"] = ""
student_flow_rates_and_graduates["Indicator Unit"] = "%"
student_flow_rates_and_graduates.rename(columns={'Provinces': 'Province'}, inplace=True)
student_flow_rates_and_graduates

student_flow_rates_and_graduates

,Province,Series Name,Indicator Value,Grade,Indicator,Year,Series Code,Sector,Sub-Sector (1),Sub-Sector (2),Source,Latitude,Longtitude,Indicator Unit
0,Banteay Meanchey,Student Flow Rates By Class Range,84.9,Primary level (Grades 1-6),Promotion,2012-2013,,Education,,,,,,%
1,Battambang,Student Flow Rates By Class Range,82.0,Primary level (Grades 1-6),Promotion,2012-2013,,Education,,,,,,%
2,Kampong Cham,Student Flow Rates By Class Range,86.4,Primary level (Grades 1-6),Promotion,2012-2013,,Education,,,,,,%
3,Kampong Chhnang,Student Flow Rates By Class Range,85.0,Primary level (Grades 1-6),Promotion,2012-2013,,Education,,,,,,%
4,Kampong Speu,Student Flow Rates By Class Range,90.6,Primary level (Grades 1-6),Promotion,2012-2013,,Education,,,,,,%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,Takeo,Student Flow Rates By Class Range,13.3,Upper Secondary level (Grades 10-12),Droput,2022-2023,,Education,,,,,,%
2516,Tbaung Khmum,Student Flow Rates By Class Range,12.8,Upper Secondary level (Grades 10-12),Droput,2022-2023,,Education,,,,,,%
2517,Whole Kingdom,Student Flow Rates By Class Range,13.7,Upper Secondary level (Grades 10-12),Droput,2022-2023,,Education,,,,,,%
2518,Urban Area,Student Flow Rates By Class Range,13.5,Upper Secondary level (Grades 10-12),Droput,2022-2023,,Education,,,,,,%


### Successful Candidates by Class

In [189]:
import pandas as pd

# Define file path
file_path = "./raw education/Public education statistic by class range.xlsx"

# Load all sheet names
sheet_names = [sheet for sheet in pd.ExcelFile(file_path).sheet_names if "dropout_trend" not in sheet.lower()]

# Initialize an empty list to store DataFrames
df_list = []

# Loop through each sheet
for sheet in sheet_names:
    # Read data
    data = pd.read_excel(file_path, sheet_name=sheet, header=[2, 3])
    data = pd.concat([data.iloc[:, 0], data.iloc[:, 10:13]], axis=1)

    # Flatten MultiIndex columns
    data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in data.columns]
    

    # Identify the correct Province column name
    province_col = [col for col in data.columns if "Provinces" in col][0]  # Automatically find it

    # Add "Series Name" column with a fixed value
    data["Series Name"] = "Successful Student By Class"

    # Melt the DataFrame
    unpivoted_data = data.melt(
        id_vars=[province_col, "Series Name"],  # Include "Series Name" in id_vars
        var_name='Grade_Indicator',
        value_name='Indicator Value'
    )

    # Split "Grade_Indicator" into "Grade" and "Indicator"
    unpivoted_data[['Indicator', 'Grade']] = unpivoted_data['Grade_Indicator'].str.rsplit('_', n=1, expand=True)

    # Rename province column back to "Province"
    unpivoted_data.rename(columns={province_col: "Provinces"}, inplace=True)

    # Drop the original "Grade_Indicator" column
    unpivoted_data.drop(columns=['Grade_Indicator'], inplace=True)

    # Add "Year" column from the sheet name
    unpivoted_data["Year"] = sheet

    # Append to list
    df_list.append(unpivoted_data)

# Combine all years into a single DataFrame
successful_candidates = pd.concat(df_list, ignore_index=True)

# Add Columns
successful_candidates["Series Code"] = "" 
successful_candidates["Sector"] = "Education" 
successful_candidates["Sub-Sector (1)"] = "" 
successful_candidates["Sub-Sector (2)"] = "" 
successful_candidates["Source"] = ""
successful_candidates["Latitude"] = ""
successful_candidates["Longtitude"] = ""
successful_candidates["Indicator Unit"] = "Number"
successful_candidates.rename(columns={'Provinces': 'Province'}, inplace=True)
successful_candidates

successful_candidates

,Province,Series Name,Indicator Value,Indicator,Grade,Year,Series Code,Sector,Sub-Sector (1),Sub-Sector (2),Source,Latitude,Longtitude,Indicator Unit
0,Banteay Meanchey,Successful Student By Class,12267.0,Successful Candidates,Grade 6,2012-2013,,Education,,,,,,Number
1,Battambang,Successful Student By Class,17147.0,Successful Candidates,Grade 6,2012-2013,,Education,,,,,,Number
2,Kampong Cham,Successful Student By Class,32285.0,Successful Candidates,Grade 6,2012-2013,,Education,,,,,,Number
3,Kampong Chhnang,Successful Student By Class,9863.0,Successful Candidates,Grade 6,2012-2013,,Education,,,,,,Number
4,Kampong Speu,Successful Student By Class,16251.0,Successful Candidates,Grade 6,2012-2013,,Education,,,,,,Number
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Takeo,Successful Student By Class,6299,Successful Candidates,Grade 12,2022-2023,,Education,,,,,,Number
836,Tbaung Khmum,Successful Student By Class,3572,Successful Candidates,Grade 12,2022-2023,,Education,,,,,,Number
837,Whole Kingdom,Successful Student By Class,80426,Successful Candidates,Grade 12,2022-2023,,Education,,,,,,Number
838,Urban Area,Successful Student By Class,24002,Successful Candidates,Grade 12,2022-2023,,Education,,,,,,Number


## Student Dropout Rate

In [190]:
# import pandas as pd

# # Define file path
# file_path = "./raw education/Public education statistic by class.xlsx"

# # Read all data
# data = pd.read_excel(file_path, sheet_name="Dropout_AllGrades_AllYears", header=[1, 2])

# # Define column index ranges (0-based index)
# column_ranges = [
#     (1, 12), (13, 24), (25, 36), (37, 48), (49, 60), 
#     (61, 72), (73, 84), (85, 96), (97, 108), (109, 120),
#     (121, 132), (133, 144)
# ]

# # Extract datasets
# datasets = []
# for start, end in column_ranges:
#     selected_data = data.iloc[:, start:end]
#     datasets.append(selected_data)
    
    
# # Initialize an empty list to store DataFrames
# df_list = []

# # Loop through each sheet
# for data in datasets:
#     # Flatten MultiIndex columns
#     data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in data.columns]
    
#     # Identify the correct Province column name
#     province_col = [col for col in data.columns if "Province" in col][0]  # Automatically find it

#     # Add "Series Name" column with a fixed value
#     data["Series Name"] = "Student Dropout Rates By Class"

#     # Melt the DataFrame
#     unpivoted_data = data.melt(
#         id_vars=[province_col, "Series Name"],  # Include "Series Name" in id_vars
#         var_name='Grade_Indicator',
#         value_name='Value'
#     )

#     # Split "Grade_Indicator" into "Grade" and "Indicator"
#     unpivoted_data[['Grade', 'Year']] = unpivoted_data['Grade_Indicator'].str.rsplit('_', n=1, expand=True)

#     # Rename province column back to "Province"
#     unpivoted_data.rename(columns={province_col: "Provinces"}, inplace=True)

#     # Drop the original "Grade_Indicator" column
#     unpivoted_data.drop(columns=['Grade_Indicator'], inplace=True)

#     # Append to list
#     df_list.append(unpivoted_data)


# # Combine all years into a single DataFrame
# student_dropout_rates_by_class = pd.concat(df_list, ignore_index=True)
# student_dropout_rates_by_class

# # # Add Columns
# student_dropout_rates_by_class["Series Code"] = "" 
# student_dropout_rates_by_class["Sector"] = "Education" 
# student_dropout_rates_by_class["Sub-Sector (1)"] = "" 
# student_dropout_rates_by_class["Source"] = ""
# student_dropout_rates_by_class["Latitude"] = ""
# student_dropout_rates_by_class["Longtitude"] = ""
# student_dropout_rates_by_class["Indicator"] = "Rate"
# student_dropout_rates_by_class["Indicator Unit"] = "%"
# # student_dropout_rates_by_class.rename(columns={'Provinces': 'Province'}, inplace=True)

# student_dropout_rates_by_class

## Dropout Rates By Range

In [191]:
# import pandas as pd

# # Define file path
# file_path = "./raw education/Public education statistic by class.xlsx"

# # Read all data
# data = pd.read_excel(file_path, sheet_name="Primary_Lower&Upper Secondary", header=[1])
# data = data.iloc[:, 1:]

# # Melting the DataFrame
# student_dropout_rates_by_range = data.melt(id_vars=["Unnamed: 1"], var_name="Year", value_name="Value")

# # Renaming the column
# student_dropout_rates_by_range.rename(columns={"Unnamed: 1": "Grade"}, inplace=True)
# student_dropout_rates_by_range["Grade"] = student_dropout_rates_by_range["Grade"].str.replace("Grades ", "", regex=False)
# # Display the result


# student_dropout_rates_by_range["Series Code"] = ""
# student_dropout_rates_by_range["Series Name"] = "Dropout Rate By Range"
# student_dropout_rates_by_range["Sector"] = "Education" 
# student_dropout_rates_by_range["Sub-Sector (1)"] = "" 
# student_dropout_rates_by_range["Source"] = ""
# student_dropout_rates_by_range["Latitude"] = ""
# student_dropout_rates_by_range["Longtitude"] = ""
# student_dropout_rates_by_range["Indicator"] = "Rate"
# student_dropout_rates_by_range["Indicator Unit"] = "%"

# student_dropout_rates_by_range


## Student_Dropout_Occupations

In [192]:
import pandas as pd

# Define file path
file_path = "./raw education/Student_Dropout_Occupations.xlsx"

# List of sheet names to process
sheet_names = ["dropout_age_6-19", "dropout_age_20-40"]

# List to store processed data for each sheet
df_list = []

# Loop over each sheet
for sheet in sheet_names:
    # Load the data from the current sheet
    data = pd.read_excel(file_path, sheet_name=sheet, header=[0, 1])
    
    # Process the data
    data = data.iloc[:, 1:]  # Remove the first column (indexing)
    data = data.head(11)  # Select the first 11 rows
    data.columns = ['_'.join(map(str, col)).strip() if isinstance(col, tuple) else str(col) for col in data.columns]  # Flatten columns
    print(data.columns)
    
    # Automatically identify the correct Province column name
    province_col = [col for col in data.columns if "Unnamed: 1_level_0_What was  ..[NAME].. 's main occupation/economic activity during the past 7 days?" in col][0]
    
    # Add "Series Name" column with a fixed value
    data["Series Name"] = "Student Dropout By Occupation"
    
    # Melt the DataFrame
    unpivoted_data = data.melt(
        id_vars=[province_col, "Series Name"],  # Include "Series Name" in id_vars
        var_name='Grade_Indicator',
        value_name='Indicator Value'
    )
    
    # Split "Grade_Indicator" into "Year" and "Indicator"
    unpivoted_data[['Year', 'Indicator']] = unpivoted_data['Grade_Indicator'].str.rsplit('_', n=1, expand=True)
    
    # Drop the original "Grade_Indicator" column
    unpivoted_data.drop(columns=['Grade_Indicator'], inplace=True)
    
    # Rename columns, ensuring any extra spaces are removed
    unpivoted_data.rename(columns=lambda x: x.strip() if isinstance(x, str) else x, inplace=True)
    unpivoted_data.rename(columns={
        "Unnamed: 1_level_0_What was  ..[NAME].. 's main occupation/economic activity during the past 7 days?": "Occupation"
    }, inplace=True)

    # Append the processed data to the list
    df_list.append(unpivoted_data)

# Concatenate all DataFrames in the list
student_dropout_rates_by_occupation = pd.concat(df_list, ignore_index=True)


student_dropout_rates_by_occupation["Series Code"] = ""
student_dropout_rates_by_occupation["Series Name"] = "Dropout Rate By Occupation"
student_dropout_rates_by_occupation["Sector"] = "Education" 
student_dropout_rates_by_occupation["Sub-Sector (1)"] = "" 
student_dropout_rates_by_occupation["Sub-Sector (2)"] = "" 
student_dropout_rates_by_occupation["Source"] = ""
student_dropout_rates_by_occupation["Latitude"] = ""
student_dropout_rates_by_occupation["Longtitude"] = ""
student_dropout_rates_by_occupation["Indicator Unit"] = "%"

student_dropout_rates_by_occupation


Index(['Unnamed: 1_level_0_What was  ..[NAME].. 's main occupation/economic activity during the past 7 days?',
       '2023_Freq.', '2023_Percent', '2021_Freq.', '2021_Percent',
       '2019_Freq.', '2019_Percent', '2017_Freq.', '2017_Percent',
       '2015_Freq.', '2015_Percent'],
      dtype='object')
Index(['Unnamed: 1_level_0_What was  ..[NAME].. 's main occupation/economic activity during the past 7 days?',
       '2023_Freq.', '2023_Percent', '2021_Freq.', '2021_Percent',
       '2019_Freq.', '2019_Percent', '2017_Freq.', '2017_Percent',
       '2015_Freq.', '2015_Percent'],
      dtype='object')


,Occupation,Series Name,Indicator Value,Year,Indicator,Series Code,Sector,Sub-Sector (1),Sub-Sector (2),Source,Latitude,Longtitude,Indicator Unit
0,"Legislations, senior officials and managers",Dropout Rate By Occupation,3.00,2023,Freq.,,Education,,,,,,%
1,Professionals,Dropout Rate By Occupation,17.00,2023,Freq.,,Education,,,,,,%
2,Technicians and associate professionals,Dropout Rate By Occupation,40.00,2023,Freq.,,Education,,,,,,%
3,Clerks,Dropout Rate By Occupation,57.00,2023,Freq.,,Education,,,,,,%
4,Service and shop and market sales workers,Dropout Rate By Occupation,383.00,2023,Freq.,,Education,,,,,,%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Craft and related trades workers,Dropout Rate By Occupation,29.44,2015,Percent,,Education,,,,,,%
216,plant and machine operators and assemblers,Dropout Rate By Occupation,5.45,2015,Percent,,Education,,,,,,%
217,Elementary occupations,Dropout Rate By Occupation,10.05,2015,Percent,,Education,,,,,,%
218,Armed forces,Dropout Rate By Occupation,0.70,2015,Percent,,Education,,,,,,%


## Merge Education Data

In [193]:
import sqlite3

combined_education_df = pd.concat([student_flow_rates, student_flow_rates_and_graduates, successful_candidates, student_dropout_rates_by_occupation], axis=0)  # For stacking vertically
combined_education_df = combined_education_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
combined_education_df['Indicator Value'] = pd.to_numeric(combined_education_df['Indicator Value'], errors='coerce')

# Rename Incorrect Value
combined_education_df['Indicator'] = combined_education_df['Indicator'].replace(
    {'Freq.': 'Frequency', 'Droput': 'Dropout'}
)

# Optionally, you can save the unpivoted data back to a new Excel file
combined_education_df.to_excel("Unpivoted_Datahub_Education.xlsx", index=False)

# Step 6: Connect to SQLite database
conn = sqlite3.connect("data.db")
table_name = "education_data"
combined_education_df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

C:\Users\Mr.Whatanak\AppData\Local\Temp\ipykernel_36816\3885273190.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_education_df = combined_education_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# Check Database data

In [194]:
import sqlite3
import pandas as pd

# Step 1: Connect to the SQLite database
conn = sqlite3.connect("data.db")

# Step 2: Fetch the list of tables in the database (to confirm the table exists)
print("List of tables in the database:")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

# Step 3: Read data from the table
table_name = "agriculture_data"

try:
    # Check if the table exists and load its content
    print(f"\nPreview of data from the '{table_name}' table:")
    data = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)
    print(data)
except Exception as e:
    print(f"Error: {e}. The table '{table_name}' might not exist.")

# Step 4: Close the connection
conn.close()


List of tables in the database:
               name
0  agriculture_data
1     economic_data
2    education_data

Preview of data from the 'agriculture_data' table:
     Series Code      Series Name       Sector             Sub-Sector (1)  \
0           None  Rice Production  Agriculture                 Production   
1           None  Rice Production  Agriculture                 Production   
2           None  Rice Production  Agriculture                 Production   
3           None  Rice Production  Agriculture                 Production   
4           None  Rice Production  Agriculture                 Production   
...          ...              ...          ...                        ...   
6174                   Rice Price  Agriculture  European Union Rice Price   
6175                   Rice Price  Agriculture  European Union Rice Price   
6176                   Rice Price  Agriculture  European Union Rice Price   
6177                   Rice Price  Agriculture  European Union Ric

In [195]:
print("Hello")

Hello
